In [1]:
import sys
sys.path.append("..")
from script.recovery import RocketParachuteCalculator
import os
from scipy.interpolate import interp1d
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys

ParaCalc = RocketParachuteCalculator()

## Manual

In [2]:
DATA_DIRECTORY = "data"
AIR_PROPERTIES_DATA_FILE_NAME = "air_values.csv"

# GENERAL DATA
MASS_OF_ROCKET = 25  # kg
AIR_TEMP_CELSIUS = 25  # Celsius

# MAIN
NUMBER_OF_MAINS = 1
NUMBER_OF_REDUNDANT = 0
MARGIN_FACTOR = 1.2
REQUIRED_DESCENT_VELOCITY = 5

# ROUND PARAMETERS
LENGTH_TO_HALF_CIRCUMFERENCE_RATIO =1
HOLE_TO_CANOPY_AREA_RATIO = 0.04
MASS_PER_UNIT_AREA = 3.5
NUMBER_OF_GORES_PANELS = [4, 4, 4, 4, 4, 12, 12, 12, 12, 12]
RATIO_OF_HEIGHT_TO_RADIUS = [1, 0.9, 0.8, 0.7, 0.6, 1, 0.9, 0.8, 0.7, 0.6]
DRAG_COEFFICIENT = 1.5

### Air Properties

In [3]:
# Get the current working directory (should be 'Notebooks')
cwd = os.getcwd()
# Go up one level to the project directory
project_dir = os.path.join(cwd, '..')
# Construct the path to the CSV file in the 'data' directory
air_csv_path = os.path.join(project_dir, DATA_DIRECTORY,
                           AIR_PROPERTIES_DATA_FILE_NAME)
# Normalize the path (optional but recommended)
air_csv_path = os.path.normpath(air_csv_path)


air_prop_df = pd.read_csv(air_csv_path)
display(air_prop_df)

,Temperature\n(K),Specific Heat,Unnamed: 2,Ratio of Specific Heats\n- k -\n(cp/cv),Dynamic Viscosity\n- ? -\n(10-5 kg/m s),Thermal Conductivity\n(10-5 kW/m K),Prandtl Number,Kinematic Viscosity1)\n- ? -\n(10-5 m2/s),Density1)\n- ? -\n(kg/m3),Diffusivity\n- ? - \n(10-6 m2/s)
0,NaN,- cp -\n(kJ/kgK),- cv -\n(kJ/kgK),NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,175.0,1.0023,0.7152,1.401,1.182,1.593,0.744,0.586,2.0170,NaN
2,200.0,1.0025,0.7154,1.401,1.329,1.809,0.736,0.753,1.7650,10.17
3,225.0,1.0027,0.7156,1.401,1.467,2.020,0.728,0.935,1.5690,NaN
4,250.0,1.0031,0.716,1.401,1.599,2.227,0.720,1.132,1.4120,15.67
5,275.0,1.0038,0.7167,1.401,1.725,2.428,0.713,1.343,1.2840,NaN
6,300.0,1.0049,0.7178,1.400,1.846,2.624,0.707,1.568,1.1770,22.07
7,325.0,1.0063,0.7192,1.400,1.962,2.816,0.701,1.807,1.0860,NaN
8,350.0,1.0082,0.7211,1.398,2.075,3.003,0.697,2.056,1.0090,29.18
9,375.0,1.0106,0.7235,1.397,2.181,3.186,0.692,2.317,0.9413,NaN


In [4]:
rename_cols = {
    'Temperature\n(K)': 'temperature',
    'Specific Heat': 'specific_heat_cp',
    'Unnamed: 2': 'specific_heat_cv',
    'Ratio of Specific Heats\n- k -\n(cp/cv)':'ratio_of_specific_heats',
    'Dynamic Viscosity\n- ? -\n(10-5 kg/m s)': 'dynamic_viscosity',
    'Thermal Conductivity\n(10-5 kW/m K)': 'thermal_conductivity',
    'Prandtl Number':'prandtl_number',
    'Kinematic Viscosity1)\n- ? -\n(10-5 m2/s)':'kinematic_viscosity',
    'Density1)\n- ? -\n(kg/m3)':'density',
    'Diffusivity\n- ? - \n(10-6 m2/s)':'diffusivity'
}

air_prop_df.rename(columns=rename_cols, inplace=True)
# Drop the first row (row 0)
air_prop_df = air_prop_df.drop(0)

# Reset the index
air_prop_df = air_prop_df.reset_index(drop=True)

In [5]:
display(air_prop_df)

,temperature,specific_heat_cp,specific_heat_cv,ratio_of_specific_heats,dynamic_viscosity,thermal_conductivity,prandtl_number,kinematic_viscosity,density,diffusivity
0,175.0,1.0023,0.7152,1.401,1.182,1.593,0.744,0.586,2.0170,NaN
1,200.0,1.0025,0.7154,1.401,1.329,1.809,0.736,0.753,1.7650,10.17
2,225.0,1.0027,0.7156,1.401,1.467,2.020,0.728,0.935,1.5690,NaN
3,250.0,1.0031,0.716,1.401,1.599,2.227,0.720,1.132,1.4120,15.67
4,275.0,1.0038,0.7167,1.401,1.725,2.428,0.713,1.343,1.2840,NaN
5,300.0,1.0049,0.7178,1.400,1.846,2.624,0.707,1.568,1.1770,22.07
6,325.0,1.0063,0.7192,1.400,1.962,2.816,0.701,1.807,1.0860,NaN
7,350.0,1.0082,0.7211,1.398,2.075,3.003,0.697,2.056,1.0090,29.18
8,375.0,1.0106,0.7235,1.397,2.181,3.186,0.692,2.317,0.9413,NaN
9,400.0,1.0135,0.7264,1.395,2.286,3.365,0.688,2.591,0.8824,36.94


## Chute Main Panel

### General Data

#### MANUAL

#### Density Interpolation 

In [6]:
air_temp_kelvin = AIR_TEMP_CELSIUS+273.15

interpolation_function = interp1d(
    air_prop_df["temperature"], air_prop_df["density"], kind='linear', fill_value="extrapolate")

# Use the interpolation function to find the density at the target temperature
density_of_air = interpolation_function(air_temp_kelvin)
print(density_of_air)

1.1849180000000001


### Mains

#### General Data

In [7]:
area_cd = ParaCalc.calc_area_cd(safety_factor=MARGIN_FACTOR,rocket_mass=MASS_OF_ROCKET,air_density=density_of_air,descent_velocity=REQUIRED_DESCENT_VELOCITY)
print(area_cd)

19.86972938211758


## Round Parameters

In [8]:
evaluated_mass_per_unit_area = MASS_PER_UNIT_AREA *0.03 # TODO IDK WHERE THE 0.03 comes from?

####  Main

In [9]:
side_length = ParaCalc.calc_gore_panel_side_length(
    area_cd=area_cd, hole_to_canopy_area_ratio=HOLE_TO_CANOPY_AREA_RATIO, drag_coefficient=DRAG_COEFFICIENT, number_of_gores_panels=NUMBER_OF_GORES_PANELS)
print(side_length)

[3.29199882 3.29199882 3.29199882 3.29199882 3.29199882 1.09733294
 1.09733294 1.09733294 1.09733294 1.09733294]


In [10]:
circumference = ParaCalc.calc_circumference(
    area_cd=area_cd, hole_to_canopy_area_ratio=HOLE_TO_CANOPY_AREA_RATIO, drag_coefficient=DRAG_COEFFICIENT,)
radius = ParaCalc.calc_radius(circumference=circumference)
print(circumference)
print(radius)

13.167995276392878
2.095751538848655


In [11]:
circular_area_of_selected = ParaCalc.calc_circular_area(
    area_cd=area_cd, drag_coefficient=DRAG_COEFFICIENT, hole_to_canopy_area_ratio=HOLE_TO_CANOPY_AREA_RATIO)
print(circular_area_of_selected)

13.798423182026097


In [12]:
round_parameters_dict = {
    'number_of_gores_panels': NUMBER_OF_GORES_PANELS,
    'ratio_of_height_to_radius': RATIO_OF_HEIGHT_TO_RADIUS
}

# Convert the dictionary to a DataFrame
round_parameters_df = pd.DataFrame(round_parameters_dict)

round_parameters_df['drag_cofficient']=DRAG_COEFFICIENT
round_parameters_df['circumference'] = circumference
round_parameters_df['radius']= radius

semi_ellipsoid_depth = ParaCalc.calc_semi_ellipsoid_depth(
    radius=radius, ratio_of_height_to_radius=round_parameters_df['ratio_of_height_to_radius'])
print(semi_ellipsoid_depth[0])
round_parameters_df['semi_ellipsoid_depth'] = semi_ellipsoid_depth[0]
display(round_parameters_df)

[2.09575154 1.88617638 1.67660123 1.46702608 1.25745092 2.09575154
 1.88617638 1.67660123 1.46702608 1.25745092]


,number_of_gores_panels,ratio_of_height_to_radius,drag_cofficient,circumference,radius,semi_ellipsoid_depth
0,4,1.0,1.5,13.167995,2.095752,2.095752
1,4,0.9,1.5,13.167995,2.095752,1.886176
2,4,0.8,1.5,13.167995,2.095752,1.676601
3,4,0.7,1.5,13.167995,2.095752,1.467026
4,4,0.6,1.5,13.167995,2.095752,1.257451
5,12,1.0,1.5,13.167995,2.095752,2.095752
6,12,0.9,1.5,13.167995,2.095752,1.886176
7,12,0.8,1.5,13.167995,2.095752,1.676601
8,12,0.7,1.5,13.167995,2.095752,1.467026
9,12,0.6,1.5,13.167995,2.095752,1.257451


In [13]:
length_of_curve = ParaCalc.calc_length_of_curve(
    radius=radius, semi_ellipsoid_depth=round_parameters_df['semi_ellipsoid_depth'],length_to_half_circumference_ratio=LENGTH_TO_HALF_CIRCUMFERENCE_RATIO)

round_parameters_df['length_of_curve'] = length_of_curve
display(round_parameters_df)

,number_of_gores_panels,ratio_of_height_to_radius,drag_cofficient,circumference,radius,semi_ellipsoid_depth,length_of_curve
0,4,1.0,1.5,13.167995,2.095752,2.095752,3.291999
1,4,0.9,1.5,13.167995,2.095752,1.886176,3.129565
2,4,0.8,1.5,13.167995,2.095752,1.676601,2.971950
3,4,0.7,1.5,13.167995,2.095752,1.467026,2.820027
4,4,0.6,1.5,13.167995,2.095752,1.257451,2.674911
5,12,1.0,1.5,13.167995,2.095752,2.095752,3.291999
6,12,0.9,1.5,13.167995,2.095752,1.886176,3.129565
7,12,0.8,1.5,13.167995,2.095752,1.676601,2.971950
8,12,0.7,1.5,13.167995,2.095752,1.467026,2.820027
9,12,0.6,1.5,13.167995,2.095752,1.257451,2.674911


In [15]:
length_of_curve_without_hole = ParaCalc.calc_length_of_curve_without_hole(
    round_parameters_df['length_of_curve'],HOLE_TO_CANOPY_AREA_RATIO)
print(length_of_curve_without_hole)

0    3.160319
1    3.004382
2    2.853072
3    2.707226
4    2.567915
5    3.160319
6    3.004382
7    2.853072
8    2.707226
9    2.567915
Name: length_of_curve, dtype: float64
